# Laboratorio 3 Data Science S10
- Arturo Argueta
- Astrid Glauser


In [1]:
import pandas as pd

In [4]:
# Carga de los conjutos de prueba y testing para ambos archivos
# Identicos al laboratorio Anterior

importacion_df = pd.read_excel('importacion.xlsx', skiprows=6)
importacion_df.columns = [
    'Fecha', 'AceitesLubricantes', 'Asfalto', 'Bunker', 'Ceras', 
    'CombustibleTurboJet', 'DieselLS', 'DieselUltraLS', 'GasLicuado', 
    'GasolinaAviacion', 'GasolinaRegular', 'GasolinaSuper', 'Kerosina', 
    'MTBE', 'Naftas', 'Petcoke', 'PetroleoCrudo', 'Solventes', 
    'Butano', 'DieselAltoAzufre', 'Orimulsion', 'PetroleoReconstituido', 
    'TotalImportacion', 'ColumnaExtra1', 'ColumnaExtra2'
]

# Convertir la columna de fechas a formato datetime
importacion_df['Fecha'] = pd.to_datetime(importacion_df['Fecha'], errors='coerce')
importacion_df = importacion_df.dropna()
#usar solo las columnas de interés ( 'GasolinaRegular', 'GasolinaSuper', 'DieselLS ,'DieselUltraLS', 'GasLicuado' y la Fecha)
importacion_df = importacion_df[['Fecha','GasolinaRegular', 'GasolinaSuper', 'DieselLS' ,'DieselUltraLS', 'GasLicuado']]
importacion_df.head()


,Fecha,GasolinaRegular,GasolinaSuper,DieselLS,DieselUltraLS,GasLicuado
0,2001-01-01,177776.50,373963.96,0.0,0.0,194065.738095
1,2001-02-01,123115.99,243091.07,0.0,0.0,170703.380952
2,2001-03-01,161726.42,312084.38,0.0,0.0,161837.371429
3,2001-04-01,127338.74,285054.89,0.0,0.0,163048.642857
4,2001-05-01,168730.19,300913.67,0.0,0.0,171518.861905


In [6]:
#Separar en Train y Test
split_date = '2021-01-01'
train_df = importacion_df[importacion_df['Fecha'] < split_date]
test_df = importacion_df[importacion_df['Fecha'] >= split_date]
print(train_df.head())
print(test_df.head())

       Fecha  GasolinaRegular  GasolinaSuper  DieselLS  DieselUltraLS  \
0 2001-01-01        177776.50      373963.96       0.0            0.0   
1 2001-02-01        123115.99      243091.07       0.0            0.0   
2 2001-03-01        161726.42      312084.38       0.0            0.0   
3 2001-04-01        127338.74      285054.89       0.0            0.0   
4 2001-05-01        168730.19      300913.67       0.0            0.0   

      GasLicuado  
0  194065.738095  
1  170703.380952  
2  161837.371429  
3  163048.642857  
4  171518.861905  
         Fecha  GasolinaRegular  GasolinaSuper    DieselLS  DieselUltraLS  \
240 2021-01-01        626537.48      642652.37  1185644.96        1499.38   
241 2021-02-01        715261.34      726508.78  1076824.67        1203.34   
242 2021-03-01        772750.01      850334.64  1345110.18       13415.05   
243 2021-04-01        634408.89     1227173.53  1250171.58       39568.59   
244 2021-05-01        669085.78      649783.14  1279017.24    